In [1]:
import tensorflow as tf

from texture.layers import DARTSEdge
from tensorflow.keras.models import Model as KerasModel
from tensorflow.keras.layers import Conv2D, BatchNormalization, Concatenate, MaxPool2D, Input, Lambda, Dense, GlobalAveragePooling2D

Using TensorFlow backend.


In [2]:
def dilation_branch(x, f_in, f_out, r, name='unnammed_conv'):
    '''
    Combination of 1x1 conv compression and 3x3 dilated conv expansion.
    '''
    x = Conv2D(f_in, (1,1), activation='relu', name=name+'_1x1_'+str(r))(x)

    #conv_init = ConvolutionAware()
    x = Conv2D(f_out, (3,3), padding='same', dilation_rate=(r,r), activation='relu', #kernel_initializer=conv_init,
               name=name+'_dilate_'+str(r))(x)

    return x

def dilation_block(x,
                   branches,
                   in_filters,
                   out_filters,
                   pool_size=None,
                   name='block'):
    '''Block with branches defined by dilation rates of `branches`.

    Parameters
    ----------
    x : Tensor, BxHxWxC
        Input Tensor
    branches : iterable(int)
        Each item defines a branch with dilation_rate=item
    in_filters : iterable or int
        Output filters for 1x1 reduction, per branch.
        If iterable, must have len == len(branches). If int, assumed all branches the same.
    out_filters : iter
        Output filters for 3x3 dilated conv, per branch.
        If iterable, must have len == len(branches). If int, assumed all branches the same.
    pool_size : tuple(int), or `None`
        If not `None`, passed to MaxPool2D on block output.
    name : str, optional
        Base name for block/branch layer names

    Returns
    -------
    Output of passing `x` through the constructed block.
    '''

    args = [branches, in_filters, out_filters]
    if all([hasattr(f, '__iter__') for f in args]):
        print([lambda f: hasattr(f, '__iter__') for f in args])
        assert len(set([len(x) for x in args])) == 1, \
            'if iterables, in_filters & out_filters must have same len as branches'
    elif isinstance(in_filters, int) and isinstance(out_filters, int):
        # if both ints
        in_filters = [in_filters] * len(branches)
        out_filters = [out_filters] * len(branches)
    else:
        raise ValueError('in_filters & out_filters must be both `iterable`, or both `int`')

    branch_outputs = []
    for rate, f_in, f_out in zip(branches, in_filters, out_filters):
        branch = dilation_branch(x, f_in, f_out, rate, name=name)
        branch_outputs.append(branch)
    #block_output = Concatenate()(branch_outputs)

    if pool_size is not None:
        branch_outputs = [MaxPool2D(pool_size=pool_size)(branch) for branch in branch_outputs]
    
    block_output = Lambda(lambda x: tf.stack(x, axis=1))(branch_outputs)

    return block_output

In [3]:
input_image = Input(shape=(224, 224, 3))

x = Conv2D(64, (3,3), padding='same', activation='relu', name='entry_conv')(input_image)

x = dilation_block(x, [1,2,3], 32, 128, pool_size=2)

x = DARTSEdge()(x)

x = dilation_block(x, [1,2,3], 32, 128, pool_size=2, name='block2')

x = DARTSEdge()(x)

x = GlobalAveragePooling2D()(x)

preds = Dense(47, activation='softmax')(x)

model = KerasModel(inputs=input_image, outputs=preds)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
entry_conv (Conv2D)             (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block_1x1_1 (Conv2D)            (None, 224, 224, 32) 2080        entry_conv[0][0]                 
__________________________________________________________________________________________________
block_1x1_2 (Conv2D)            (None, 224, 224, 32) 2080        entry_conv[0][0]                 
__________________________________________________________________________________________________
block_1x1_

In [4]:
model.layers[-3].get_weights()[0].squeeze()

array([1., 1., 1.], dtype=float32)

In [5]:
from texture.datasets import DTDDataset

In [6]:
dtd = DTDDataset('/home/administrator/Dropbox/benchmark/dtd')
dtd.load_or_generate_data()

/home/administrator/anaconda3/envs/core-dev/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [7]:
from texture.datasets import DatasetSequence

trainseq = DatasetSequence(dtd.X_train, dtd.y_train, 12)
testseq = DatasetSequence(dtd.X_train, dtd.y_train, 12)

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
hist = model.fit_generator(
    trainseq,
    epochs=10,
    validation_data=testseq,
    shuffle=False
)

Epoch 1/10
